Load control data.

In [1]:
from modules.helper_functions_tsp import (read_index, 
                                          find_run_stats,
                                          find_distances_array,
                                          find_problem_size,
                                          )

from modules.AWS_helper_functions_tsp import (define_parameters, 
                                              cost_fn_fact, 
                                              update_parameters_using_gradient, 
                                              create_initial_rotations,
                                              vqc_circuit, 
                                              )

from classes.MyDataLogger import MyDataLogger, MySubDataLogger

import numpy as np
import copy
import time
from pathlib import Path

from modules.config import(CONTROL_DIR, 
                           CONTROL_FILE, 
                           ENCODING, 
                           )

from braket.jobs import hybrid_job
from braket.jobs import save_job_result

In [2]:
control_path = Path(CONTROL_DIR).joinpath(CONTROL_FILE)
control_dict = read_index(control_path, ENCODING)
print(f'Reading control data from {control_path}')
print(control_dict)
EMPTY = ' '

Reading control data from control/control_parameters.csv
{0: {'quantum': 'TRUE', 'locations': '12', 'slice': '1.0', 'shots': '1024', 'mode': '2', 'iterations': '250', 'gray': 'FALSE', 'hot_start': 'FALSE', 'gradient_type': 'SPSA', 'formulation': 'original', 'layers': '1', 'std_dev': '', 'lr': '', 'weight_decay': '', 'momentum': '', 'alpha': '0.602', 'big_a': '25', 'c': '0.314', 'eta': '0.1', 'gamma': '0.101', 's': '0.5', 'noise': 'FALSE'}}


In [3]:
len_dict = len(control_dict)
if len_dict != 1:
    raise Exception(f'The control dictionary has {len_dict} items, only one should be submitted to a quantum device currently')

Instantiate datalogger:

In [4]:
datalogger = MyDataLogger()

In [5]:
def validate_sdl(sdl):
    if sdl.quantum != True:
        raise Exception(f'Incorrect value for {sdl.quantum=}.  It should be True to run with AWS')
    if sdl.noise != False:
        raise Exception(f'Incorrect value for {sdl.noise=}.  It should be False to run with AWS')
    if sdl.hot_start != False:
        raise Exception(f'Incorrect value for {sdl.hot_start=}.  It should be False to run with AWS')

In [6]:
@hybrid_job(device="local:braket/default",
            include_modules=['modules', 'classes',],
            dependencies = ['graycode', 'numpy',]
           )
def run_hybrid_algorithm(locations:int,
                         average_slice:float,
                         shots:int,
                         mode:int,
                         iterations:int,
                         gray:bool,
                         hot_start:bool,
                         gradient_type:str,
                         formulation:str,
                         layers:int,
                         alpha:float,     
                         big_a:float,
                         c:float,
                         eta:float,
                         gamma:float,
                         s:float,
                         noise:bool,                                 
                         params: list,
                         rots: np.ndarray,
                         cost_fn,
                         print_results=True,
                         ):
    
    index_list, cost_list, lowest_list, gradient_list, average_list, parameter_list = [], [], [], [], [],[]
    
    qc = vqc_circuit(qubits,
                     mode,
                     noise,
                     layers,
                     params, 
                     )

    (index_list, 
    cost_list,
    lowest_list,
    gradient_list,
    average_list,
    parameter_list) = update_parameters_using_gradient(
                                     locations,
                                     average_slice,
                                     shots,
                                     mode,
                                     iterations,
                                     gray,
                                     hot_start,
                                     gradient_type,
                                     formulation,
                                     layers,
                                     alpha,
                                     big_a,
                                     c,
                                     eta,
                                     gamma,
                                     s,
                                     noise,
                                     params,
                                     rots,
                                     cost_fn,
                                     qc,
                                     )
    
    items, hits, misses = cost_fn.report_cache_stats()
        
    results = {
            'index_list': index_list,
            'cost_list': cost_list,
            'lowest_list': lowest_list,
            'average_list': average_list,
            'cache_items': items,
            'cache_hits' : hits,
            'cache_misses': misses,
             }

    save_job_result(results)
    return results         # Optional, but safe as fallback

In [7]:
def update_sub_data_logger(sdl, result):
    sdl.index_list   = result['index_list']
    sdl.sliced_list  = result['cost_list']
    sdl.lowest_list  = result['lowest_list']
    sdl.average_list = result['average_list']
    sdl.cache_items = result['cache_items']
    sdl.cache_hits = result['cache_hits']
    sdl.cache_misses = result['cache_misses']
    sdl.average_list_all.append(sdl.average_list)
    sdl.lowest_list_all.append(sdl.lowest_list)
    sdl.sliced_cost_list_all.append(sdl.sliced_list)
    sdl.best_dist_found, sdl.iteration_found = find_run_stats(sdl.lowest_list)
    sdl.save_results_to_csv()
    sdl.save_detailed_results()
    sdl.save_plot()
    return()

## Main loop

In [ ]:
sdl_list = []
for i, control_items in control_dict.items():
    t0 = time.time()
    data_dict = dict(control_items)
    sdl_list.append(MySubDataLogger(runid=datalogger.runid))
    sdl_list[i].update_constants_from_dict(data_dict)
    validate_sdl(sdl_list[i])
    #can't send sdl to hybrid jobs, so need to send item by item
    locations = sdl_list[i].locations
    average_slice = sdl_list[i].slice
    shots = sdl_list[i].shots
    mode = sdl_list[i].mode
    iterations = sdl_list[i].iterations
    gray = sdl_list[i].gray
    hot_start = sdl_list[i].hot_start
    gradient_type = sdl_list[i].gradient_type
    formulation = sdl_list[i].formulation
    layers = sdl_list[i].layers
    alpha = sdl_list[i].alpha
    big_a = sdl_list[i].big_a
    c = sdl_list[i].c
    eta = sdl_list[i].eta
    gamma = sdl_list[i].gamma
    s = sdl_list[i].s
    noise = False

    #calculate information
    qubits = find_problem_size(sdl_list[i])
    distance_array, best_dist = find_distances_array(locations)
    sdl_list[i].qubits = qubits                                                
    num_params = sdl_list[i].calculate_parameter_numbers()
    sdl_list[i].best_dist = best_dist
    sdl_list[i].hot_start_dist = 'n/a'
    
    params = define_parameters(qubits, mode, num_params)
    init_rots = create_initial_rotations(qubits, 
                                         mode, 
                                         layers,
                                         bin_hot_start_list=[],
                                         )
    
    rots = copy.deepcopy(init_rots)
    average_list_all = []
    lowest_list_all = []
    sliced_cost_list_all = []
    
    cost_fn = cost_fn_fact(locations,
                       gray,
                       formulation,
                       distance_array,
                      )
    
    job = run_hybrid_algorithm(locations,
                                   average_slice,
                                   shots,
                                   mode,
                                   iterations,
                                   gray,
                                   hot_start,
                                   gradient_type,
                                   formulation,
                                   layers,
                                   alpha,     
                                   big_a,
                                   c,
                                   eta,
                                   gamma,
                                   s,
                                   noise,                                 
                                   params,
                                   rots,
                                   cost_fn,
                                   )
    

    #results = job.result()
    t1 = time.time()
    sdl_list[i].elapsed = t1-t0
    
    update_sub_data_logger(sdl_list[i], job.result())


SubDataLogger instantiated.  Run ID = 20260216-19-02-11 - 19-02-11
Reading distance data
